In [ ]:
import numpy as np
import pandas as pd

def generate_data(strategy, size):
    np.random.seed(0)
    if strategy == 'unique':
        return np.arange(size)
    elif strategy == 'repeated':
        return np.random.choice(size, size)
    elif strategy == 'low_skewed':
        prob = np.exp(-np.linspace(0, 1, size) * 4)  # heavily skewed towards smaller numbers
        prob = prob / prob.sum()  # Normalize
        return np.random.choice(np.arange(size), size, p=prob)
    elif strategy == 'high_skewed':
        prob = np.exp(np.linspace(0, 1, size) * 4)  # heavily skewed towards larger numbers
        prob = prob / prob.sum()  # Normalize
        return np.random.choice(np.arange(size), size, p=prob)
    elif strategy == 'middle_focused':
        return np.clip(np.random.normal(size/2, size/10, size).astype(int), 0, size-1)
    elif strategy == 'random':
        return np.random.choice(1000, size)

def generate_synthetic_data_v5():
    sizes = [1000, 1200, 800, 900, 1100, 850]
    strategies = ['unique', 'repeated', 'low_skewed', 'unique', 'high_skewed', 'random']

    tables = []
    for i, size in enumerate(sizes):
        j_id1_strategy = strategies[i]
        j_id2_strategy = np.random.choice(strategies)

        data = {
            'j_id1': generate_data(j_id1_strategy, size),
            'j_id2': generate_data(j_id2_strategy, size)
        }
        table = pd.DataFrame(data)
        tables.append(table)

    return tables

def pandas_equi_join(table1, table2, key1, key2):
    """
    Perform an equi-join between two tables using pandas merge.
    """
    # Perform the join
    result = table1.merge(table2, left_on=key1, right_on=key2, how='inner', suffixes=('', '_right'))

    # Drop columns with '_right' suffix
    columns_to_drop = [col for col in result.columns if '_right' in col]
    result.drop(columns=columns_to_drop, inplace=True)

    # Retain only the first two columns
    result = result.iloc[:, :2]

    return result


def pandas_multi_way_join(tables, join_keys):
    """
    Perform a multi-way join over a list of tables using pandas.

    Args:
    - tables (list): A list of DataFrames representing the tables to be joined.
    - join_keys (list): A list of tuple pairs representing the join keys for consecutive tables.

    Returns:
    - DataFrame: The result of the multi-way join.
    """
    if len(tables) != len(join_keys) + 1:
        raise ValueError("Number of join keys should be one less than the number of tables.")

    result = tables[0]
    cumulative_size = 0
    for i in range(1, len(tables)):
        result = pandas_equi_join(result, tables[i], join_keys[i-1][0], join_keys[i-1][1])
        print(f"Intermediate Size after join with table {i}: {len(result)}")
        cumulative_size += len(result)

    print(f"\nCumulative Intermediate Size: {cumulative_size}")
    print(f"Final Size: {len(result)}\n")

    return result

def filter_relation(table, column_name, value_range):
    """
    Filter rows of a relation based on a specified range for a given column.

    Args:
    - table (pd.DataFrame): The relation (table) to be filtered.
    - column_name (str): The column on which the filter will be applied.
    - value_range (tuple): A tuple of two values representing the range (inclusive).

    Returns:
    - pd.DataFrame: The filtered relation.
    """
    lower_bound, upper_bound = value_range
    return table[(table[column_name] >= lower_bound) & (table[column_name] <= upper_bound)]


def example_basic(tables):
    '''
    Basic operations
    '''

    # Filter
    # Use filter to filter tables[0].j_id1 in (100, 200)
    # SELECT * FROM tables[0] WHERE tables[0].j_id1 > 100 AND  tables[0].j_id1 < 200
    #print("\033[31m" + "This text is red" + "\033[0m")
    print("\033[32m" + "Filter operator" + "\033[0m")
    filtered_table = filter_relation(tables[0], 'j_id1', (100, 200))
    print(filtered_table.head())
    print("\n" + "="*40 + "\n")

    # Join
    # Join two relations on certain key
    # SELECT * FROM tables[0] join tables[1] ON 'j_id1'
    print("\033[32m" + "Join operator" + "\033[0m")
    joined_result = pandas_equi_join(tables[0], tables[1], 'j_id1', 'j_id1')
    print(joined_result.head())
    print("\n" + "="*40 + "\n")

    # Multi-way join
    # Join Order 1: tables[3] -> tables[2] -> tables[1] -> tables[0]
    # Multi-way join has build-in performance counter
    print("\033[32m" + "Multi-way join operator" + "\033[0m")
    print("Join Order 2: tables[3] -> tables[2] -> tables[1] -> tables[0]\n")
    result_order2 = pandas_multi_way_join(
        [tables[3], tables[2], tables[1], tables[0]],
        [('j_id1', 'j_id1'), ('j_id1', 'j_id1'), ('j_id1', 'j_id1')]
    )
    print(result_order2.head())
    print("\n" + "="*40 + "\n")


    # Join Order 2: tables[1] -> tables[3] -> tables[0] -> tables[2]
    print("\033[32m" + "Multi-way join with different join order" + "\033[0m")
    print("Join Order 3: tables[1] -> tables[3] -> tables[0] -> tables[2]\n")
    result_order3 = pandas_multi_way_join(
        [tables[1], tables[3], tables[0], tables[2]],
        [('j_id1', 'j_id1'), ('j_id1', 'j_id1'), ('j_id1', 'j_id1')]
    )
    print(result_order3.head())
    print("\n" + "="*40 + "\n")




'''
Assignment specified queries
'''

# Query 1 - Implement the following query using provided functions (10pts)
#
#   SELECT * FROM tables[4] A, tables[2] B
#   WHERE A.j_id1 = B.j_id1
#
def q1(tables):
    # Your implementations

    pass



# Query 2 - Implement the following query using provided functions (10pts)
#
#   SELECT * FROM tables[4] A
#   WHERE A.j_id1 within (100, 500)
#
def q2(tables):
    # Your implementations

    pass



# Query 3 - Implement the following query using provided functions
#           and conduct filter push-down for query optimization (10pts)
#
#   SELECT * FROM tables[4] A, tables[2] B
#   WHERE A.j_id1 = B.j_id1 AND A.j_id1 within (100, 500)
#
#   Hint: Order of filter and joins matters.

def q3(tables):
    # Your implementations

    pass



# Query 4 - Implement the following query using pandas_multi_way_join(),
#           you need to find the optimal join order such that the Cumulative
#           Intermediate Size should beat the provided baseline (10pts)
#
#   SELECT * FROM tables[0] A, tables[1] B, tables[3] C
#   WHERE A.j_id1 = B.j_id1 AND B.j_id1 = C.j_id1
#
#   Hint: What join is non-expanding?
#   Baseline: 1812 (culumative size, directly calculated by pandas_multi_way_join)
#
def q4 (tables):
    # Your implementations

    pass



# Query 5 - Implement the following query using pandas_multi_way_join(), and
#           filter_relation() you need to find the optimal join order such that
#           the Cumulative Intermediate Size should beat the provided baseline (15pts)
#
#   SELECT * FROM tables[5] A, tables[2] B, tables[4] C
#   WHERE A.j_id1 = B.j_id1 AND B.j_id1 = C.j_id1 AND
#     A.j_id2 within (100, 500) AND
#     B.j_id1 within (50, 600) AND
#     C.j_id1 within (200, 700)
#
#   Hint: (1)Rule-based optimization; (2) Join key distributions
#   Baseline: 81 (culumative size, directly calculated by pandas_multi_way_join)
#
def q5(tables):
  # Your implementations

  pass



# Query 6 - Implement the following query using pandas_multi_way_join(),
#           you need to find the optimal join order (15pts)
#
#           Grading: Full credits if you beat 3 randomly selected join orders
#                    75% credits if you beat 2 of them
#                    50% credits if you beat only one
#
#   SELECT * FROM tables[0] A, tables[1] B, tables[2] C, tables[4] D
#   WHERE A.j_id1 = B.j_id1 AND B.j_id1 = C.j_id1 AND C.j_id1 = D.j_id1
#
#   Hint: (1)Non-expanding joins (2)Join key distributions
#
def q6(tables):
  # Your implementations

  pass



# Query 7 - Implement the following query using pandas_multi_way_join(),
#           you need to find the optimal join order (15pts)
#
#           Grading: Full credits if you beat 3 randomly selected join orders
#                    75% credits if you beat 2 of them
#                    50% credits if you beat only one
#
#   SELECT * FROM tables[1] A, tables[2] B, tables[3] C, tables[4] D, tables[5] E
#   WHERE A.j_id1 = B.j_id1 AND B.j_id1 = C.j_id1 AND C.j_id1 = D.j_id1 AND D.j_id1 = E.j_id1
#
def q7(tables):
  # Your implementations

  pass



# Query 8 - Implement the following query using pandas_multi_way_join(), and filter_relation()
#           you need to find the optimal join order (15pts)
#
#           Grading: Full credits if you beat 3 randomly selected join orders
#                    75% credits if you beat 2 of them
#                    50% credits if you beat only one
#
#   SELECT * FROM tables[0] A, tables[1] B, tables[2] C, tables[4] D, tables[5] E
#   WHERE A.j_id1 = B.j_id1 AND B.j_id1 = C.j_id1 AND C.j_id1 = D.j_id1 AND D.j_id1 = E.j_id1 AND
#         A.j_id2 within (100, 500) AND E.j_id2 within (200, 500)
#
def q8(tables):
  # Your implementations

  pass


'''
Execution
'''
# Data generation, do not alter this
tables = generate_synthetic_data_v5()

# Examples
example_basic(tables)

# Execute your queries
q1(tables)

q2(tables)

q3(tables)

q4(tables)

q5(tables)

q6(tables)

q7(tables)

q8(tables)

Filter operator
     j_id1  j_id2
100    100    905
101    101    685
102    102    924
103    103    990
104    104    665


Join operator
   j_id1  j_id2
0      0    853
1      0    853
2      3    852
3      3    852
4      3    852


Multi-way join operator
Join Order 2: tables[3] -> tables[2] -> tables[1] -> tables[0]

Intermediate Size after join with table 1: 800
Intermediate Size after join with table 2: 778
Intermediate Size after join with table 3: 778

Cumulative Intermediate Size: 2356
Final Size: 778

   j_id1  j_id2
0      0    768
1      0    768
2      0    768
3      0    768
4      0    768


Multi-way join with different join order
Join Order 3: tables[1] -> tables[3] -> tables[0] -> tables[2]

Intermediate Size after join with table 1: 911
Intermediate Size after join with table 2: 911
Intermediate Size after join with table 3: 778

Cumulative Intermediate Size: 2600
Final Size: 778

   j_id1  j_id2
0    684    684
1    277    277
2    277    277
3    599    599
4  